In [1]:
!mkdir -p ~/.kaggle
from google.colab import drive
from google.colab import files
# Upload your kaggle.json API key
uploaded = files.upload()

drive.mount('/content/drive')

Saving kaggle.json to kaggle.json
Mounted at /content/drive


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:

!mv kaggle.json ~/.kaggle/
!mkdir -p ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json


In [4]:
import os
import shutil
os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive/Machine-learning'


In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Conv2D, Conv2DTranspose
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
import kaggle
import pandas as pd
print("Import ok!")

Import ok!


In [8]:
data = pd.read_csv('/content/drive/MyDrive/Machine-learning/hmnist_28_28_RGB.csv')

In [9]:
y = data['label']
x = data.drop(columns = ['label'])
len(x )

10015

In [10]:
tabular_data = pd.read_csv('/content/drive/MyDrive/Machine-learning/HAM10000_metadata.csv')

In [11]:
classes = {4: ('nv', ' melanocytic nevi'), 6: ('mel', 'melanoma'), 2 :('bkl', 'benign keratosis-like lesions'), 1:('bcc' , ' basal cell carcinoma'), 5: ('vasc', ' pyogenic granulomas and hemorrhage'), 0: ('akiec', 'Actinic keratoses and intraepithelial carcinomae'),  3: ('df', 'dermatofibroma')}
classes

{4: ('nv', ' melanocytic nevi'),
 6: ('mel', 'melanoma'),
 2: ('bkl', 'benign keratosis-like lesions'),
 1: ('bcc', ' basal cell carcinoma'),
 5: ('vasc', ' pyogenic granulomas and hemorrhage'),
 0: ('akiec', 'Actinic keratoses and intraepithelial carcinomae'),
 3: ('df', 'dermatofibroma')}

#Oversampling

In [12]:
from imblearn.over_sampling import RandomOverSampler
oversample = RandomOverSampler()
x,y  = oversample.fit_resample(x,y)

In [13]:
x = np.array(x).reshape(-1,28,28,3)
print('Shape of X :',x.shape)

Shape of X : (46935, 28, 28, 3)


#Standardization and Splitting Data

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

x_standardized = (x - np.mean(x)) / np.std(x)
# Reshape the standardized 'x' if it has more than two dimensions
num_samples = x_standardized.shape[0]
x_flattened = x_standardized.reshape(num_samples, -1)

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(x_flattened, y, test_size=0.2, random_state=1)

In [15]:
# Standardize features (optional but recommended)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Generator network

In [16]:
def build_generator(latent_dim):
    model = tf.keras.Sequential()
    model.add(Dense(128 * 7 * 7, activation="relu", input_dim=latent_dim))
    model.add(Reshape((7, 7, 128)))
    model.add(Conv2DTranspose(128, kernel_size=4, strides=2, padding="same", activation="relu"))
    model.add(Conv2DTranspose(64, kernel_size=4, strides=2, padding="same", activation="relu"))
    model.add(Conv2DTranspose(1, kernel_size=7, padding="same", activation="sigmoid"))
    return model

# Discriminator network


In [17]:
def build_discriminator(input_shape):
    model = tf.keras.Sequential()
    model.add(Conv2D(64, kernel_size=3, strides=2, input_shape=input_shape, padding="same", activation="relu"))
    model.add(Conv2D(128, kernel_size=3, strides=2, padding="same", activation="relu"))
    model.add(Flatten())
    model.add(Dense(1, activation="sigmoid"))
    return model

# GAN

In [18]:

def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = tf.keras.Sequential([generator, discriminator])
    return model

# Build and compile the models

In [19]:

latent_dim = 100
input_shape = (28, 28, 1)

generator = build_generator(latent_dim)
discriminator = build_discriminator(input_shape)
gan = build_gan(generator, discriminator)

discriminator.compile(loss="binary_crossentropy", optimizer=Adam(), metrics=["accuracy"])
gan.compile(loss="binary_crossentropy", optimizer=Adam())


In [25]:
# Training loop
batch_size = 64
epochs = 10000

In [28]:
for epoch in range(epochs):
    # Train discriminator
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    real_imgs = X_train[idx]
    fake_imgs = generator.predict(np.random.randn(batch_size, latent_dim))
    d_loss_real = discriminator.train_on_batch(real_imgs, np.ones((batch_size, 1)))
    d_loss_fake = discriminator.train_on_batch(fake_imgs, np.zeros((batch_size, 1)))
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Train generator
    noise = np.random.randn(batch_size, latent_dim)
    g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))

    # Print progress and save generated images
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, D Loss: {d_loss[0]}, G Loss: {g_loss}")
        generated_imgs = generator.predict(np.random.randn(16, latent_dim))
        plt.figure(figsize=(4, 4))
        for i in range(generated_imgs.shape[0]):
            plt.subplot(4, 4, i + 1)
            plt.imshow(generated_imgs[i, :, :, 0], cmap="gray")
            plt.axis("off")
        plt.tight_layout()
        plt.show()

2/2 [==============================] - 0s 5ms/step


TypeError: ignored